In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

Using TensorFlow backend.


In [11]:
seed = 21

# Load & Pre-process the Data

In [12]:
from keras.datasets import cifar10

In [13]:
# load the data from keras datasets
(X_train, y_train), (X_test,y_test) = cifar10.load_data()

In [14]:
# Pre-process the data (normalize the input data)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [15]:
# One hot encode the values (use Numpy command to_categorical() from np_utils)
y_train = np_utils.to_categorical(y_train)
y_test  = np_utils.to_categorical(y_test)
class_num = y_test.shape[1]

# Design and Model Creation

In [16]:
# First layer of our model
model = Sequential()

In [17]:
model.add(Conv2D(32,(3,3), input_shape=X_train.shape[1:], padding='same')) #pooling
model.add(Activation('elu')) #activation

In [18]:
# Make a dropout layer to prevent overfitting
# Eliminating connections between the layouts
model.add(Dropout(0.2)) #drop 0.2 * 100% of the existing connection

In [19]:
# Batch Normalization:
# Ensuring that the network always creates activations with the same distribution that we desire
model.add(BatchNormalization())

In [20]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('elu'))

In [21]:
# Pooling layer: help the image classifier to be more robust in terms of identifying relevant pattern
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

In [22]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

# Flatten the Data

In [23]:
# Flatten the output and eliminate some of the connections again
model.add(Flatten())
model.add(Dropout(0.2))

In [24]:
# Densely Connected Layer
model.add(Dense(256, kernel_constraint=maxnorm(3)))
model.add(Activation('elu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
    
model.add(Dense(128, kernel_constraint=maxnorm(3)))
model.add(Activation('elu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

First Densely Connected Layer: Specifying the number of neurons in the dense layer.
kernel_constraint is used to regularized the data as it learns (it also helps with reducing overfitting)

In [25]:
model.add(Dense(class_num))
model.add(Activation('softmax')) #softmax activation select which neurons have the highest probability as output

In [26]:
epochs = 25
optimizer = 'adam'

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [28]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)       

# Model Fit

In [29]:
np.random.seed(seed)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=64)

Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 386s 8ms/step - loss: 1.4222 - accuracy: 0.4955 - val_loss: 0.9898 - val_accuracy: 0.6470
Epoch 2/25
50000/50000 [==============================] - 376s 8ms/step - loss: 0.9637 - accuracy: 0.6600 - val_loss: 0.8316 - val_accuracy: 0.7144
Epoch 3/25
50000/50000 [==============================] - 375s 8ms/step - loss: 0.8102 - accuracy: 0.7170 - val_loss: 0.7998 - val_accuracy: 0.7213
Epoch 4/25
50000/50000 [==============================] - 381s 8ms/step - loss: 0.7260 - accuracy: 0.7453 - val_loss: 0.6960 - val_accuracy: 0.7548
Epoch 5/25
50000/50000 [==============================] - 377s 8ms/step - loss: 0.6738 - accuracy: 0.7624 - val_loss: 0.6751 - val_accuracy: 0.7646
Epoch 6/25
50000/50000 [==============================] - 375s 8ms/step - loss: 0.6283 - accuracy: 0.7784 - val_loss: 0.6236 - val_accuracy: 0.7833
Epoch 7/25
50000/50000 [==============================] - 376s

In [32]:
# Model Evaluation
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 0.5090310754776001 / Test accuracy: 0.8305000066757202
